# Fix pathing

In [1]:
import sys


sys.path.append("../..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))))


# Imports

In [3]:
import plotter
import datahandler

import matplotlib.pyplot
import numpy as np
import pandas as pd
import seaborn as sns
import IPython.display


# Constants

In [4]:
FOLDER_NAME = "2024_05_06_20_13_39_NONE"

FOLDER_PATH = os.path.join(os.path.dirname(constants.PROJECT_DIRECTORY_PATH), "Simulator", "data", FOLDER_NAME)


In [5]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(False, False, True)


Cleaning dataset: 100%|██████████| 2/2 [00:00<?, ?it/s]


Loading dataset: 100%|██████████| 2/2 [00:03<00:00,  1.54s/it]


# Methods

In [6]:
def load_csv(fileName):
    df = pd.read_csv(os.path.join(FOLDER_PATH, fileName + ".csv"))

    response_time_cols = [
        'duration_incident_creation',
        'duration_resource_appointment',
        'duration_resource_preparing_departure',
        'duration_dispatching_to_scene'
    ]
    df['total_response_time'] = df[response_time_cols].sum(axis=1)

    return df


In [7]:
def print_results(df: pd.DataFrame):
    # Define the criteria for response times
    criteria = {
        ('A', True): 12 * 60,
        ('A', False): 25 * 60,
        ('H', True): 30 * 60,
        ('H', False): 40 * 60
    }

    # Function to calculate compliance for each group
    def calculate_compliance(group, triage, urban):
        limit = criteria.get((triage, urban))
        if limit is not None:
            return (group['total_response_time'] < limit).mean()
        return None

    # Calculate statistics and compliance for each group
    results = []
    for (triage, urban), group in df.groupby(['triage_impression_during_call', 'urban']):
        mean = group['total_response_time'].mean()
        median = group['total_response_time'].median()
        compliance = calculate_compliance(group, triage, urban)
        results.append({
            'Triage': triage,
            'Urban': urban,
            'Mean (sec)': mean,
            'Median (sec)': median,
            'Compliance': compliance
        })

    stats = pd.DataFrame(results)

    # Convert mean and median to minutes
    stats['Mean (min)'] = (stats['Mean (sec)'] / 60)
    stats['Median (min)'] = (stats['Median (sec)'] / 60)
    stats.drop(columns=['Mean (sec)', 'Median (sec)'], inplace=True)

    # Map urban values to Yes/No
    stats['Urban'] = stats['Urban'].map({True: 'Yes', False: 'No'})
    
    # Sort values
    stats.sort_values(by=["Urban", "Triage"], ascending=[False, True], inplace=True)
    
    # Display the DataFrame
    formatted_stats = stats.style.format({
        'Mean (min)': "{:.2f}",
        'Median (min)': "{:.2f}",
        'Compliance': "{:.2%}"
    }).hide(axis='index')
    IPython.display.display(formatted_stats)


In [8]:
def boxplot_time_at_steps_modified(
    dataframe: pd.DataFrame,
    triage_impression: str = None
):
    title = "Time Taken At Each Step of the Incident"

    if triage_impression is not None:
        # Filter the dataframe without overwriting the original one
        temp_df = dataframe[dataframe["triage_impression_during_call"] == triage_impression].copy()
        title += f" ({triage_impression})"
    else:
        # Use the original dataframe if no triage_impression filter is applied
        temp_df = dataframe.copy()

    steps = {
        "Creating Incident": "duration_incident_creation",
        "Appointing Resource": "duration_resource_appointment",
        "Resource to Start Task": "duration_resource_preparing_departure",
        "Dispatching to Scene": "duration_dispatching_to_scene",
        "At Scene": "duration_at_scene",
        "Dispatching to Hospital": "duration_dispatching_to_hospital",
        "At Hospital": "duration_at_hospital"
    }

    # Calculating durations for each step
    plot_data = [(temp_df[duration_column][temp_df[duration_column] > 0] / 60) for step, duration_column in steps.items()]

    # Plotting
    matplotlib.pyplot.figure(figsize=(8, 4))
    matplotlib.pyplot.boxplot(plot_data[::-1], labels=list(steps.keys())[::-1], vert=False, patch_artist=True, showfliers=False)
    matplotlib.pyplot.title(title)
    matplotlib.pyplot.xlabel("Time in Minutes")
    matplotlib.pyplot.xticks()
    matplotlib.pyplot.show()


In [9]:
def boxplot_time_at_steps(
    historic_dataframe: pd.DataFrame,
    simulated_dataframe: pd.DataFrame,
    bounds: tuple[str, str] = None,
    triage_impressions: list[str] = ["A", "H", "V1"],
):
    historic_df = historic_dataframe.copy()
    simulated_df = simulated_dataframe.copy()
    
    # filter historic data by time frame
    if bounds is not None:
        start_bound, end_bound = pd.to_datetime(bounds[0]), pd.to_datetime(bounds[1])
        historic_df = historic_df[(historic_df['time_call_received'] >= start_bound) & (historic_df['time_call_received'] <= end_bound)]

    # calculate duration at each stage in minutes (simulated dataframe already has this calculated)
    historic_steps = {
        "duration_incident_creation": ("time_call_received", "time_incident_created"),
        "duration_resource_appointment": ("time_incident_created", "time_resource_appointed"),
        "duration_resource_preparing_departure": ("time_resource_appointed", "time_ambulance_dispatch_to_scene"),
        "duration_dispatching_to_scene": ("time_ambulance_dispatch_to_scene", "time_ambulance_arrived_at_scene"),
        "duration_at_scene": ("time_ambulance_arrived_at_scene", "time_ambulance_dispatch_to_hospital", "time_ambulance_available"),
        "duration_dispatching_to_hospital": ("time_ambulance_dispatch_to_hospital", "time_ambulance_arrived_at_hospital"),
        "duration_at_hospital": ("time_ambulance_arrived_at_hospital", "time_ambulance_available")
    }

    for step, times in historic_steps.items():
        if len(times) == 3:
            historic_df.loc[historic_df[times[1]].isna(), step] = (historic_df[times[2]] - historic_df[times[0]]).dt.total_seconds() / 60
            historic_df.loc[~historic_df[times[1]].isna(), step] = (historic_df[times[1]] - historic_df[times[0]]).dt.total_seconds() / 60
        else:
            historic_df[step] = (historic_df[times[1]] - historic_df[times[0]]).dt.total_seconds() / 60
    
    # convert seconds to minutes and replace zeros in simulated data to nan
    for step in historic_steps.keys():
        simulated_df[step] /= 60

    simulated_df[["duration_dispatching_to_hospital", "duration_at_hospital"]] = simulated_df[["duration_dispatching_to_hospital", "duration_at_hospital"]].replace(0, np.nan)

    # plot data
    matplotlib.pyplot.figure(figsize=(8, 8))

    data = []
    positions = []
    labels = []
    colors = []
    stripes = []

    i = 0
    for step in historic_steps.keys():
        labels.append(f"{step}")
        for triage in triage_impressions:
            for df, stripe, alpha in zip([historic_df, simulated_df], [False, True], [1.00, 0.65]):
                data.append(df[df['triage_impression_during_call'] == triage][step].dropna())

                labels.append("")

                if (triage == "A"):
                    colors.append([1.00, 0.37, 0.28, alpha])
                elif (triage == "H"):
                    colors.append([0.12, 0.56, 1.00, alpha])
                else:
                    colors.append([0.20, 0.80, 0.20, alpha])

                stripes.append(stripe)

                positions.append(i)
                i += 0.75
        labels.pop()
        i += 1

    bplot = matplotlib.pyplot.boxplot(
        data[::-1],
        labels=labels[::-1],
        positions=positions,
        vert=False,
        patch_artist=True,
        showfliers=True
    )

    for patch, color, stripe in zip(bplot["boxes"], colors[::-1], stripes[::-1]):
        patch.set_facecolor(color)

    matplotlib.pyplot.title("Time Taken At Each Step of the Incident")
    matplotlib.pyplot.xlabel("Time in Minutes")
    matplotlib.pyplot.xticks()
    matplotlib.pyplot.show()


In [10]:
def table_results():
    def calculate_compliance(df, urban_filter=None):
        if df is None:
            return None

        # Criteria for both triage types in both urban and rural settings
        criteria = {
            ('A', True): 12 * 60,  # 12 minutes for triage 'A' in urban areas
            ('A', False): 25 * 60,  # 25 minutes for triage 'A' in rural areas
            ('H', True): 30 * 60,  # 30 minutes for triage 'H' in urban areas
            ('H', False): 40 * 60  # 40 minutes for triage 'H' in rural areas
        }

        # Initialize counters
        total_compliant_cases = 0
        total_cases = 0

        # Check necessary columns exist
        for urban in [True, False]:
            if urban_filter is not None and urban != urban_filter:
                continue
            for triage in ['A', 'H']:
                filtered_df = df[(df['triage_impression_during_call'] == triage) & (df['urban'] == urban)]
                limit = criteria.get((triage, urban))
                if not filtered_df.empty:
                    # Count compliant cases for this triage type
                    compliant_cases = filtered_df['total_response_time'] < limit
                    total_compliant_cases += compliant_cases.sum()
                    total_cases += len(filtered_df)

        if total_cases > 0:
            # Calculate overall compliance rate across both triage types
            overall_compliance = total_compliant_cases / total_cases
            return overall_compliance
        else:
            return None  # Return None if there are no cases to evaluate

    # Factors to test
    factors = [0.5, 1.0, 1.5]

    results = []

    for factor in factors:
        filename = f"events_numIncidentsFactor={factor:.6f}"
        df = load_csv(filename)
        results.append({
            "Factor": factor,
            "Compliance": calculate_compliance(df),
            "Urban Compliance": calculate_compliance(df, urban_filter=True),
            "Rural Compliance": calculate_compliance(df, urban_filter=False)
        })

    # Create DataFrame
    results_df = pd.DataFrame(results)

    return results_df



# Main

In [11]:
table_results()


,Factor,Compliance,Urban Compliance,Rural Compliance
0,0.5,0.134831,0.113924,0.300000
1,1.0,0.219251,0.229814,0.153846
2,1.5,0.167832,0.177165,0.093750


In [12]:
print_results(load_csv("events_numIncidentsFactor=0.500000"))


Triage,Urban,Compliance,Mean (min),Median (min)
A,Yes,7.14%,23.46,21.84
H,Yes,16.22%,45.38,42.15
V1,Yes,nan%,115.66,104.77
A,No,20.00%,38.95,28.78
H,No,40.00%,43.79,42.25
V1,No,nan%,199.51,213.01


In [13]:
print_results(load_csv("events_numIncidentsFactor=1.000000"))


Triage,Urban,Compliance,Mean (min),Median (min)
A,Yes,16.67%,22.67,19.57
H,Yes,28.92%,41.33,34.97
V1,Yes,nan%,108.73,98.23
A,No,16.67%,37.67,35.22
H,No,12.50%,67.04,69.32
V1,No,nan%,141.02,147.72


In [14]:
print_results(load_csv("events_numIncidentsFactor=1.500000"))


Triage,Urban,Compliance,Mean (min),Median (min)
A,Yes,11.21%,58.74,25.92
H,Yes,22.45%,80.97,45.85
V1,Yes,nan%,141.33,109.77
A,No,10.00%,51.57,48.13
H,No,8.33%,58.84,61.97
V1,No,nan%,253.45,202.45
